In [ ]:
import pandas as pd
import string
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np
import torch
from torch.autograd import Variable

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27448 entries, 0 to 27447
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27447 non-null  object
 1   sentiment  27448 non-null  object
dtypes: object(2)
memory usage: 429.0+ KB


In [ ]:
def text_processor():
    df['text'] = df['text'].apply(lambda x : remove_stopwords(str(x)))
    df['text'] = df['text'].apply(lambda x : str(x).lower().translate(str.maketrans('','',string.punctuation)))
    df['text'] = df['text'].apply(lambda x : ''.join([i for i in x if not i.isdigit()]))
    # df['text_list'] = df['text_list'].apply(lambda x : np.asarray(x))

text_processor()

In [ ]:
df.head()

,text,sentiment
0,oh marly im sorry i hope soon,neutral
1,playing ghost online interesting the new updat...,positive
2,cleaning house family comming later today,neutral
3,gotta restart i thought win supposed end cons...,neutral
4,see wat i mean bout follw friidays its called ...,neutral


In [ ]:
def tokenize_corpus(text):
    tokens = str(text).split()
    return tokens

df['tokenized_text'] = df['text'].apply(lambda x : tokenize_corpus(x))

In [ ]:
df.head()

,text,sentiment,tokenized_text
0,oh marly im sorry i hope soon,neutral,"[oh, marly, im, sorry, i, hope, soon]"
1,playing ghost online interesting the new updat...,positive,"[playing, ghost, online, interesting, the, new..."
2,cleaning house family comming later today,neutral,"[cleaning, house, family, comming, later, today]"
3,gotta restart i thought win supposed end cons...,neutral,"[gotta, restart, i, thought, win, supposed, en..."
4,see wat i mean bout follw friidays its called ...,neutral,"[see, wat, i, mean, bout, follw, friidays, its..."


In [ ]:
vocabulary = []

for sentence in df['tokenized_text']:
    for element in sentence:
        if element not in vocabulary:
            vocabulary.append(element)

In [ ]:
word2idx = {w : idx for(idx, w) in enumerate(vocabulary)}
idx2word = {idx : w for(idx, w) in enumerate(vocabulary)}

In [ ]:
vocabulary_size = len(vocabulary)

In [ ]:
window_size = 2
idx_pairs = []

for sentence in df['tokenized_text']:
    indices = [word2idx[word] for word in sentence]
    for center_word_pos in range(len(indices)):
        for w in range(-window_size, window_size+1):
            context_word_pos = center_word_pos + w
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))
            
idx_pairs - np.array(idx_pairs)

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [ ]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).type(dtype)
    x[word_idx] = 1.0
    return x

In [ ]:
embedding_dims = 5
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).type(dtype), requires_grad =True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).type(dtype), requires_grad=True).type(dtype)

num_epochs = 100
learning_rate = 0.001
num = 1

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).type(dtype)
        y_true = Variable(torch.from_numpy(np.array([target])).type(torch.cuda.LongTensor))
        
        z1 = torch.matmul(W1, x).type(dtype)
        z2 = torch.matmul(W2, z1).type(dtype)
        
        log_softmax = torch.nn.functional.log_softmax(z2, dim=0)
        
        loss = torch.nn.functional.nll_loss(log_softmax.view(1, -1), y_true).type(dtype)
        loss_val += loss.data
        loss.backward()

        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
        
        if num % 1000 == 0:
            print(f'num/1000 : {num}')
        num += 1

    if epo % 10 == 0:
        print(f'Loss at epo {epo} : {loss_val/len(idx_pairs)}')

Streaming output truncated to the last 5000 lines.
num/1000 : 25042000
num/1000 : 25043000
num/1000 : 25044000
num/1000 : 25045000
num/1000 : 25046000
num/1000 : 25047000
num/1000 : 25048000
num/1000 : 25049000
num/1000 : 25050000
num/1000 : 25051000
num/1000 : 25052000
num/1000 : 25053000
num/1000 : 25054000
num/1000 : 25055000
num/1000 : 25056000
num/1000 : 25057000
num/1000 : 25058000
num/1000 : 25059000
num/1000 : 25060000
num/1000 : 25061000
num/1000 : 25062000
num/1000 : 25063000
num/1000 : 25064000
num/1000 : 25065000
num/1000 : 25066000
num/1000 : 25067000
num/1000 : 25068000
num/1000 : 25069000
num/1000 : 25070000
num/1000 : 25071000
num/1000 : 25072000
num/1000 : 25073000
num/1000 : 25074000
num/1000 : 25075000
num/1000 : 25076000
num/1000 : 25077000
num/1000 : 25078000
num/1000 : 25079000
num/1000 : 25080000
num/1000 : 25081000
num/1000 : 25082000
num/1000 : 25083000
num/1000 : 25084000
num/1000 : 25085000
num/1000 : 25086000
num/1000 : 25087000
num/1000 : 25088000
num/1000 

In [ ]:
len(idx_pairs)

663816